# Tutorial 1 - Basics

Welcome to the  *IBM SMF Explorer* Basics Tutorial.
You will learn how to get started with the *IBM SMF Explorer* framework and access SMF data.

The tutorial has cells containing code. When you come across such a code cell you should execute it by selecting it and pressing ``Ctrl``+``Enter``.
Feel free to change the code, but keep in mind that other parts of the tutorial might be affected. 


## Getting started

To start working with the *IBM SMF Explorer*, you need to import the ``smfexplorer`` package:

In [ ]:
import smfexplorer

This tutorial was created for *SMF Explorer* version **1.0.0**.
To check the version, execute the following command: ``smfexplorer.__version__``.

In [ ]:
smfexplorer.__version__

All requests in the *SMF Explorer* are managed by an object called a **Context**.
A Context represents a connection to one or multiple SMF dumps/datasets and manages the state of all requests dispatched against it.
Creating separate contexts allows you to run the same requests against different SMF dumps in an easy manner.

The Context can be created by calling the ``new_context()`` function of the ``smfexplorer`` module.
If multiple datasets are defined (i.e., provided as arguments to the ``new_context()``), the *SMF Explorer* will execute requests against all of them and concatenate the data in the order the names were specified.

To separately fetch the data from different datasets, you can create as many Context objects as you like. 

> **Note**: if you are working with two or more datasets that were created as a result of the dump-dataset switch (i.e., one dataset is a continuation of another), we recommend assigning these datasets to one Context. 
If you want to work with datasets that contain different SMF records, we advise creation of multiple Contexts for better performance.   

In [ ]:
DATASET = "YOUR.SMF.DATA"
ctx = smfexplorer.new_context(DATASET)

# Multiple datasets assigned to one Context instance:
# ctx2 = smfexplorer.new_context('YOUR.SMF.DATA1','YOUR.SMF.DATA2')

*IBM SMF Explorer* provides utility functions that can help you to understand your input dataset. 

You can check if a dataset is available:

In [ ]:
print(smfexplorer.check_dataset(DATASET))
print(smfexplorer.check_dataset("WRONG.SMF.DATA"))

You can fetch a list of SMF types/subtypes available in your SMF dataset using ```get_available_records()```function. 

To get the list of types you can run ```get_available_types()``` function and for subtypes - ```get_available_subtypes()```.

In [ ]:
# get available types
ctx.get_available_types()

In [ ]:
# get available subtypes for the SMF type 71
ctx.get_available_subtypes(70)

In [ ]:
# get a complete SMF record overview
ctx.get_available_records()

## Fetching Data

The standard way to fetch the data using the *IBM SMF Explorer* is by defining the list of SMF fields you want to request.
Therefore, the Framework provides you with predefined definitions for many SMF fields.
To access those definitions you need to import them from the ``smfexplorer.fields`` module.
Fields are defined in classes that correspond to the SMF type and subtype.


In the following example, we import all fields for the SMF type 70 subtype 1 record.

> **Note**: the following naming scheme: SMF**XX**S**Y**, where *XX* represents record type and *Y* its subtype 

In [ ]:
from smfexplorer.fields import SMF70S1

After importing from the fields module, you can access SMF field documentation by pressing ``shift``+``tab``.
To test this out for the field ``lpar_name``, just place the cursor behind the ``SMF70S1.lpar_name`` line below and press the key combination.
Alternatively, you can use the ipython **?** syntax to get help.

In [ ]:
# select this cell and press shift+tab to see the documentation
SMF70S1.lpar_busy

In [ ]:
?SMF70S1.lpar_name

Such information is provided on each field.
Feel free to have a look into the properties available.

To make working with SMF data easier, additional **virtual fields** are provided.
Virtual fields are derived from the SMF fields by the *IBM SMF Explorer*.
For example, the ``ziip_boost`` field shows whether zIIP Boost was active.

The virtual field ``ziip_boost`` uses  ``fla``(SMF70FLA) field to extract the information.

In [ ]:
?SMF70S1.ziip_boost

### Request() Method

To use the field definitions for data fetching, you can use the ``request()`` method of a Context.
You need to provide an array of fields you would like to request.
To trigger the request call ``.run()``.
When the request succeeds, it returns a pandas DataFrame.

In [ ]:
ctx.request(
    [
        SMF70S1.timestamp,
        SMF70S1.sid,
        SMF70S1.lpar_name,
        SMF70S1.system_name,
        SMF70S1.sysplex_name,
        SMF70S1.lpar_system_name,
        SMF70S1.lpar_number,
        SMF70S1.lpar_cpu_count,
    ]
).run()

The *IBM SMF Explorer* is a Fluent-API.
That means you chain methods together to configure a request.
You have seen the most basic form of such a chain: ``request().run()``. The ``request()`` method can start a chain and the ``run()`` method ends it.
You will learn about other methods that can be used in the chain later and in other tutorials.

Note, that you cannot combine fields arbitrarily in a request.
Not all fields are compatible, because they may originate from different structures and cannot be displayed in a single table in a logical/useful way.

The following request, for example, tries to combine SMF 70 Subtype 1 and SMF 72 Subtype 3 data into one table and **throws an exception**.
In general, you cannot combine fields of different record types.

**WARNING**: the following code causes an error

In [ ]:
from smfexplorer.fields import SMF72S3

ctx.request([SMF70S1.timestamp, SMF70S1.sid, SMF72S3.utilization_total]).run()

#### Raw fields

The *IBM SMF Explorer* applies different transformations to the raw SMF data before returning it to the user.
Sometimes you might want to disable the post-processing of some fields to get the original values.

You can use the `raw` property of each field to see raw SMF values.
What `raw` returns depends on the type of the field.
For normal fields, post-processing is disabled.
For virtual fields, the raw value of the source field will be returned.

In the example below, you can see how `raw` fetches the `class_goal_type` value without the *IBM SMF Explorer* post-processing.

In [ ]:
ctx.request(
    [
        SMF72S3.timestamp,
        SMF72S3.sid,
        SMF72S3.class_name,
        SMF72S3.class_period,
        SMF72S3.class_goal_type,
        SMF72S3.class_goal_type.raw,
    ]
).run()

### Using Sample Requests
To make common tasks easier, *IBM SMF Explorer* comes with a collection of predefined requests.
The data can be fetched from related fields without listing them individually.
These requests can be found in the ``samples`` property of any context.
In other tutorials, you will be shown how to create and register your own samples.

To fetch the same information as above (i.e., in the first successful ``ctx.request()`` call), we can use the ``lpar_information()`` sample request.

In [ ]:
ctx.samples.lpar_information().run()

To fetch additional fields together with a sample, the ``run()`` method has a named parameter called ``display``.
You can provide an array of fields to this parameter just like in the ``request()`` method.

In [ ]:
ctx.samples.lpar_information().run(display=[SMF70S1.capactiy_group_name])

# Some available Samples

## Samples for SMF70S1:

``lpar_information()`` --  fields from SMF70S1 on LPARs

``processor_information()`` --  fields from SMF70S1 on processors

## Samples for SMF72S3:

``smf_72_03_sample()`` -- fields for SMF72 subtype 3 analysis


## Samples for SMF99S1:
    
``p_utilization()`` --  CP, zIIP, zAAP and total utilization
    
``rg_capping()`` -- Resource Group and Tenant Resource Group capping information
    
``smf_99_01_sample()`` --  commonly used SMF 99 subtype 1 data


## Samples for SMF99S2:
 

``srv_service()`` -- service Class service consumption for CP, zIIP and zAAP

``smf_99_02_sample()`` -- commonly used SMF 99 Subtype 2 data

## Samples for SMF99S12:

``hiper_dispatch()`` -- hiper dispatch information per processor type

## Samples for SMF99S14:

``topology()`` -- topology information per processor
